In [1]:
import os
from multiprocessing import Pool
from typing import Optional

import networkx as nx
import numpy as np

from src.vrp_study.data_loader import parse_data
from src.vrp_study.configs import ModelConfig
from src.vrp_study.data_model import Tariff, Cargo, Node
from src.vrp_study.routing_manager import RoutingManager
import pickle

from vrp_study.initial_solution.pdptw_solution_builder import SolutionBuilder
from vrp_study.managers.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from vrp_study.or_tools.model import VRPModelFactory, VRPSolver

In [2]:
def func(du, dv):
    return np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2)


def calc(data):
    u, du, p2coordinate = data
    return {(u, v): func(du, dv) for v, dv in p2coordinate.items()}


def build_routing_manager(
        depo: Node,
        cargos: list[Cargo],
        tariff: Tariff,
        p: Optional[Pool] = None
) -> RoutingManager:
    p2coordinates = {}

    p2coordinates.update({
        crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
    })
    p2coordinates[depo.id] = depo.coordinates

    if p is not None:
        res = list(p.imap_unordered(calc, [(u, du, p2coordinates) for u, du in p2coordinates.items()]))
    else:
        res = [calc((u, du, p2coordinates)) for u, du in p2coordinates.items()]

    distance_matrix = {}
    time_matrix = {}
    for r in res:
        for k, v in r.items():
            distance_matrix[k] = v
            time_matrix[k] = v

    routing_manager = PDRoutingManagerBuilder(
        distance_matrix=distance_matrix,
        time_matrix=time_matrix
    )

    routing_manager.add_cargos(cargos)
    routing_manager.add_tariff(tariff)

    routing_manager.add_depo(depo)

    routing_manager = routing_manager.build()
    return routing_manager

In [3]:
def graph_to_real_node(g: nx.DiGraph, rm: RoutingManager) -> nx.DiGraph:
    res = nx.DiGraph()
    for u, v, d in g.edges(data=True):
        a = rm.get_node(u).routing_node.id
        b = rm.get_node(v).routing_node.id
        res.add_edge(a, b, **d)
    return res

In [4]:
def get_file_name(path: str, name: str) -> str:
    i = 0
    while os.path.exists(f'{path}/{i}_{name}'):
        i += 1
    return f'{path}/{i}_{name}'

In [5]:
import gc


def rms_calc(data):
    # Включить отладку утечек
    gc.set_debug(gc.DEBUG_LEAK)

    routing_manager, benchmark_type, name, cfg = data
    routing_manager: RoutingManager = routing_manager
    model_factory = VRPModelFactory.get_pdptw_model_factory(routing_manager)

    sb = SolutionBuilder(
        solver=VRPSolver(
            model_config=ModelConfig(max_execution_time_minutes=1, max_solution_number=50)
        ),
        model_factory=model_factory
    )

    p = f'../data/Li & Lim benchmark/graphs/{benchmark_type}/graph_{name}.pkl'

    if not os.path.exists(p):
        cg = sb.generate_full_graph(routing_manager)
        with open(p, 'wb') as f:
            pickle.dump(graph_to_real_node(cg, routing_manager), f)

    sol = VRPSolver(
        model_config=ModelConfig(max_execution_time_minutes=10),
        initial_solution_builder=sb,
        callback=None
    ).solve(model_factory.build_model(routing_manager))
    sols = []

    for s in sol[0]:
        if len(s) > 0:
            sols.append([routing_manager.nodes()[i].routing_node.id for i in s[1:-1]])
    with open(get_file_name(f'../data/Li & Lim benchmark/parsed_solutions/{benchmark_type}',
                            f'{cfg.ls_type}_{cfg.first_solution_type}_{name}'), 'wb') as f:
        pickle.dump((sols, routing_manager), f)
    del routing_manager, benchmark_type, name, sol, sols

    # Принудительный сбор мусора
    gc.collect()

    # Показать "мусорные" объекты
    print(gc.garbage)  # Если gc.DEBUG_LEAK включен, здесь могут быть утечки



In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from tqdm import tqdm
from loguru import logger as log

log.remove()
NUM_WORKERS = 3
MAX_SIZE = max(10, NUM_WORKERS)  # создаем столько менеджеров. потом в парралель NUM_WORKERS  решают

arr_ls = [
    routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC,
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
    routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT
]

arr_first = [
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC,
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
    routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,
    # routing_enums_pb2.FirstSolutionStrategy.,
    # routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC,
    # routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC

]

with Pool(NUM_WORKERS) as p:
    for benchmark_type in ['pdp_100']:  #os.listdir('../data/Li & Lim benchmark'):
        rms = []
        for name in tqdm(os.listdir(f'../data/Li & Lim benchmark/benchmarks/{benchmark_type}')):
            depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/benchmarks/{benchmark_type}/{name}')
            for ls in arr_ls:
                for first in arr_first:
                    rm = build_routing_manager(depo, cargos, tariff, p)
                    cfg = ModelConfig(
                        max_execution_time_minutes=1
                    )
                    rms.append((rm, benchmark_type, name, cfg))
                    if len(rms) >= MAX_SIZE:
                        rr = list(p.imap_unordered(rms_calc, rms))
                        rms = []
        break
    if len(rms) > 0:
        rr = list(p.imap_unordered(rms_calc, rms))
        rms = []

  2%|▏         | 1/56 [00:00<00:54,  1.00it/s]gc: collectable <DiGraph 0x7fbc45005040>
gc: collectable <dict 0x7fbc4505dd40>
gc: collectable <OutEdgeView 0x7fbc8b80b840>
gc: collectable <AdjacencyView 0x7fbc451cfa30>
gc: collectable <builtin_function_or_method 0x7fbc8b80aac0>
gc: collectable <dict 0x7fbc45368f40>
gc: collectable <dict 0x7fbc45080840>
gc: collectable <dict 0x7fbc453940c0>
gc: collectable <DiGraph 0x7fbc45537a70>
gc: collectable <dict 0x7fbc4e0ac200>
gc: collectable <dict 0x7fbc4505d1c0>
gc: collectable <dict 0x7fbc4e0ad500>
gc: collectable <OutEdgeView 0x7fbc4520d1d0>
gc: collectable <dict 0x7fbc4e0ad4c0>
gc: collectable <AdjacencyView 0x7fbc45200820>
gc: collectable <dict 0x7fbc4e0ad440>
gc: collectable <dict 0x7fbc4e0ac100>
gc: collectable <builtin_function_or_method 0x7fbc45453740>
gc: collectable <dict 0x7fbc4e0ad3c0>
gc: collectable <DiGraph 0x7fbc4533f1a0>
gc: collectable <dict 0x7fbc4505dd40>
gc: collectable <dict 0x7fbc45b9ecc0>
gc: collectable <dict 0x7fbc4e0ac

In [ ]:

# for file_name in os.listdir('../data/graphs'):
#     solution_name = file_name.replace('_cg', '')
#     with open(f'../data/results/{solution_name}', 'rb') as f:
#         res = pickle.load(f)
#     sols, rm = res
#     rm: RoutingManager = rm
#     assert check_solution(rm, sols)

In [ ]:

for file_name in os.listdir('../data/graphs'):
    solution_name = file_name.replace('_cg', '')
    print(file_name, solution_name)
    with open(f'../data/graphs/{file_name}', 'rb') as f:
        cg = pickle.load(f)
    with open(f'../data/results/{solution_name}', 'rb') as f:
        res = pickle.load(f)
    sols, rm = res
    rm: RoutingManager = rm
    cg: nx.DiGraph = cg
    # print(cg.edges())

    edges2remove = set()

    for u, v, d in cg.edges(data='length'):
        if d > 7:
            edges2remove.add((u, v))
    cg.remove_edges_from(edges2remove)

    for sol in sols:
        cargo_path_set = set()
        cargo_path = []
        for i in range(len(sol)):
            pdp = rm.nodes()[sol[i]].pdp_id
            if pdp not in cargo_path_set:
                cargo_path_set.add(pdp)
                cargo_path.append(pdp)
        print(sol)
        print(cargo_path)

        for i in range(len(cargo_path) - 1):
            p1 = cargo_path[i]
            p2 = cargo_path[i + 1]
            n1 = rm.get_pick_up_and_delivery_nodes()[p1][0]
            n2 = rm.get_pick_up_and_delivery_nodes()[p2][0]
            e = cg.edges()[n1, n2]
            print(e)

In [ ]:
cg

In [ ]:
from matplotlib import pyplot as plt

edge_values = np.array([d['length'] for u, v, d in cg.edges(data=True)])
edge_values = edge_values[edge_values < 7]
plt.hist(edge_values, bins=100)

In [ ]:
edge_values.min(), edge_values.max()